In [ ]:
!py -m pip install open3d trimesh numpy scikit-learn

In [32]:
#open3d to visualize
import trimesh

load_mesh = trimesh.load_mesh('pie_escaneado.stl')
#open3d to visualize
import open3d as o3d
def visualize_mesh(mesh):
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(mesh.vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(mesh.faces)
    o3d_mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([o3d_mesh])

visualize_mesh(load_mesh)

In [48]:
# !pip install trimesh
path = "foot.stl"

import trimesh
import numpy as np

# Load the mesh
mesh = trimesh.load(path)

# plot cloud point
cloud = mesh.sample(1000)  # Sample 1000 points from the mesh
# %pip install open3d
import open3d as o3d

def plot_cloud(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud)
    # Create coordinate frame
    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=20, origin=[0, 0, 0])
    o3d.visualization.draw_geometries([pcd, axis])

plot_cloud(cloud)

# mesh.vertices -= mesh.center_mass

from sklearn.decomposition import PCA

# centered_points = mesh.vertices - np.mean(mesh.vertices, axis=0)  # Center the points around the origin
centered_points = mesh.vertices - mesh.center_mass  # Center the points around the mesh center of mass

pca = PCA(n_components=3)
pca.fit(centered_points)
basis_vectors = pca.components_  # shape (3, 3)

# Transform the points to the PCA basis
transformed_points = centered_points @ basis_vectors.T  # shape (n, 3)
# Plot the transformed points
plot_cloud(transformed_points)

In [49]:
plot_cloud(transformed_points)

In [35]:

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(centered_points)


In [39]:
pcd

PointCloud with 55460 points.

In [41]:
type(transformed_points)

trimesh.caching.TrackedArray

In [50]:
# unir puntos rotados y pasar a formato stl como nueva malla rotada
rotated_mesh = trimesh.Trimesh(vertices=transformed_points, faces=mesh.faces)
# Save the rotated mesh to a new STL file
rotated_mesh.export('rotated_foot.stl')
# Visualize the rotated mesh
visualize_mesh(rotated_mesh)


In [ ]:
import trimesh
import numpy as np

# === CONFIGURACIÓN ===
ruta_stl = "/rotated_foot.stl"
umbral_normal = 0.7  # Coseno del ángulo con el eje Z negativo
porcentaje_z = 15  # Parte inferior del modelo (en %)

# === CARGAR MALLA ===
print("Cargando el modelo...")
mesh = trimesh.load(ruta_stl)
if not isinstance(mesh, trimesh.Trimesh):
    mesh = mesh.dump(concatenate=True)

print(f"Número de caras en la malla original: {len(mesh.faces)}")

# === CALCULAR NORMALES ===
face_normals = mesh.face_normals
z_down = np.array([0, 0, -1])
dot_products = face_normals @ z_down  # Producto punto entre normal y -Z

# === FILTRO POR DIRECCIÓN DE NORMAL ===
indices_por_normal = np.where(dot_products > umbral_normal)[0]
print(f"Caras que apuntan hacia abajo: {len(indices_por_normal)}")

""" # === FILTRO POR POSICIÓN Z ===
centros_caras = mesh.triangles_center
z_coords = centros_caras[:, 2]
umbral_z = np.percentile(z_coords, porcentaje_z)
indices_por_z = np.where(z_coords < umbral_z)[0]
print(f"Caras en el {porcentaje_z}% inferior del modelo: {len(indices_por_z)}")

# === COMBINAR AMBOS FILTROS ===
indices_finales = np.intersect1d(indices_por_normal, indices_por_z)
print(f"Caras que cumplen ambos criterios: {len(indices_finales)}")
"""

indices_finales = indices_por_normal

# === CREAR NUEVA MALLA ===
if len(indices_finales) == 0:
    print("⚠️ No se encontraron caras que cumplan los criterios. Ajusta los umbrales.")
else:
    nueva_malla = trimesh.Trimesh(
        vertices=mesh.vertices, faces=mesh.faces[indices_finales], process=False
    )

    # === GUARDAR O VISUALIZAR ===
    nueva_malla.export("planta_pie.stl")
    print("✅ Planta del pie exportada como 'planta_pie.stl'")
#    nueva_malla.show()

Cargando el modelo...


TypeError: _dump() got an unexpected keyword argument 'concatenate'

In [ ]:
# con la planta del pie, extruyo hacia abajo
extrusion_depth = 10  # Profundidad de extrusión en mm
extruded_mesh = nueva_malla.extrude(
    direction=[0, 0, -1], distance=extrusion_depth, process=False
)

#aplano a 0 el eje z de la extrusión
extruded_mesh.vertices[:, 2] = 0

#visualizar el nuevo mesh extruido
visualize_mesh(extruded_mesh)